In [ ]:
import pandas as pd
import time
from bs4 import BeautifulSoup
from splinter import Browser
executable_path = {'executable_path': 'C:/Users/markg/chromedriver/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
def scrape_html_soup(seed_url, soup=True, click_css=''):
    # Visit the seed url
    browser.visit(seed_url)
    time.sleep(1)
    if click_css != '':
        browser.find_by_css(click_css).first.click()
        time.sleep(1)
    html = browser.html
    if soup: ## Scrape page into soup
        return BeautifulSoup(html, 'html.parser')
    else:
        return html

In [ ]:
def scrape_mars_news():
    seed_url = 'https://mars.nasa.gov/news/'
    soup = scrape_html_soup(seed_url)
    article_title = soup.find('div', class_='content_title').text
    article_paragraph = soup.find('div', class_='article_teaser_body').text
    return {'articleTitle': article_title, 'articleParagraph': article_paragraph}
scraped_data = scrape_mars_news()
print(scraped_data['articleTitle'])
print(scraped_data['articleParagraph'])

In [ ]:
def scrape_featured_mars_image():
    seed_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/'
    soup = scrape_html_soup(seed_url, click_css='div.carousel_container div.carousel_items a.button')
    landing_page_url = seed_url[:seed_url.find('/',8)]
    featured_image_url = soup.find('img', class_='fancybox-image')['src']
    featured_image_url = f'{landing_page_url}{featured_image_url}'
    featured_image_url = featured_image_url.replace('medium', 'large').replace('ip', 'hires')
    return {'featuredImageUrl': featured_image_url}
scraped_data = scrape_featured_mars_image()
scraped_data['featuredImageUrl']

In [ ]:
def scrape_mars_weather_tweet():
    seed_url = 'https://twitter.com/marswxreport?lang=en'
    soup = scrape_html_soup(seed_url)
    mars_weather_tweet = soup.find('div', attrs={'class': 'tweet', 'data-name': 'Mars Weather'})\
        .find('p', attrs={'class': 'tweet-text'}).text
    return {'marsWeatherTweet': mars_weather_tweet}
scraped_data = scrape_mars_weather_tweet()
scraped_data['marsWeatherTweet']

In [ ]:
def scrape_mars_facts_df():
    seed_url = 'http://space-facts.com/mars/'
    html = scrape_html_soup(seed_url, soup=False)
    mars_facts_df = pd.read_html(html, attrs = {'id': 'tablepress-mars'})[0]
    mars_facts_df.columns = ['Measurement', 'Value']
    mars_facts_df['Measurement'] = [measurement[:-1] for measurement in mars_facts_df['Measurement']]
    mars_facts_html = mars_facts_df.to_html()
    return {'marsFactsHtml': mars_facts_html}
scraped_data = scrape_mars_facts_df()
scraped_data['marsFactsHtml']